In [57]:
import pandas as pd
import numpy as np
from scipy import stats
import os

In [58]:
path = 'D:\\FILESTEX\\2019_Probabilistic_guarantees_portfolio\\NYSE_2'
os.chdir(path)
fnames=['ahp.csv','alcoa.csv','amerb.csv','coke.csv','dow.csv','dupont.csv',\
        'ford.csv','ge.csv','gm.csv','hp.csv','ibm.csv','inger.csv','jnj.csv','kimbc.csv',\
       'merck.csv','mmm.csv','morris.csv','pandg.csv','schlum.csv']
names=['ahp','alcoa','amerb','coke','dow','dupont','ford','ge','gm','hp','ibm','inger','jnj','kimbc',\
       'merck','mmm','morris','pandg','schlum']
stocks=pd.concat([pd.read_csv(fname,header=None).drop(0,axis=1).rename(columns={1:fname}) for fname in fnames], axis=1)
stocks.columns=names
stocks.head()

,ahp,alcoa,amerb,coke,dow,dupont,ford,ge,gm,hp,ibm,inger,jnj,kimbc,merck,mmm,morris,pandg,schlum
0,1.01515,1.02765,1.04183,1.00637,1.00847,1.01983,1.00000,1.00000,1.01026,1.01935,1.00429,1.01357,0.99683,1.05340,1.03148,1.03377,1.01495,1.00775,1.01176
1,1.01493,1.04036,0.98905,1.00475,1.00840,1.00833,1.00157,1.02187,0.99746,1.01266,0.99573,1.00446,1.00318,1.00461,1.00898,1.00251,1.00000,1.00192,1.01938
2,1.00000,0.97629,0.97786,0.98583,0.99722,0.99449,0.98116,0.97860,0.98219,0.98125,0.98571,0.99556,0.95873,0.98165,0.98043,0.95990,0.97218,0.98656,0.97338
3,1.02451,1.00662,1.02642,1.01917,0.99443,1.00693,1.02720,1.00795,0.98705,1.00637,1.01522,1.00000,1.01325,0.98131,1.01089,1.03655,0.99663,1.00778,1.00000
4,1.03100,0.98465,1.00368,1.00313,1.02801,1.00413,1.04361,1.00394,1.00525,1.03165,1.02427,1.01563,1.00654,1.02381,1.01077,0.99496,0.98649,1.01158,1.01563


In [59]:
# path = 'D:\\FILESTEX\\2019_Probabilistic_guarantees_portfolio\\NYSE_1'
# os.chdir(path)
# fnames=['ahp.csv','alcoa.csv','amerb.csv','arco.csv','coke.csv','comme.csv','dow.csv','dupont.csv',\
#         'espey.csv','exxon.csv','fisch.csv','ford.csv','ge.csv','gm.csv','gte.csv','gulf.csv','hp.csv',\
#         'ibm.csv','inger.csv','iroqu.csv','jnj.csv','kimbc.csv','kinar.csv','kodak.csv','luken.csv','meico.csv',\
#        'merck.csv','mmm.csv','mobil.csv','morris.csv','pandg.csv','pills.csv','schlum.csv','sears.csv',\
#         'sherw.csv','tex.csv']
# names=['ahp','alcoa','amerb','arco','coke','comme','dow','dupont',\
#         'espey','exxon','fisch','ford','ge','gm','gte','gulf','hp',\
#         'ibm','inger','iroqu','jnj','kimbc','kinar','kodak','luken','meico',\
#        'merck','mmm','mobil','morris','pandg','pills','schlum','sears',\
#         'sherw','tex']
# stocks=pd.concat([pd.read_csv(fname,header=None).drop(0,axis=1).rename(columns={1:fname}) for fname in fnames], axis=1)
# stocks.columns=names
# print(stocks.shape)
# stocks.head(10)

In [60]:
# path = 'D:\\FILESTEX\\2019_Probabilistic_guarantees_portfolio\\NYSE'
# os.chdir(path)
# f = open('NYSE.txt')
# data=[]
# for line in f:
#     data.append([float(x) for x in line.split()])
# f.close()
# names=['ahp','alcoa','amer_brands','arco','coke','comm_metals','dow_chem','Dupont','espey_manufacturing',\
#        'Exxon','fischbach','ford','ge','gm','gte','gulf','hp','ibm','ingersoll','Iroquois','jnj','kimb-clark',\
#        'kin_ark','Kodak','lukens','mei_corp','merck','mmm','mobil','morris','p_and_g','Pillsbury',\
#        'schlum','sears','sher_will','texaco']
# stocks=pd.DataFrame(data,columns=names) 
# print(stocks.shape)
# stocks.head()

In [61]:
N=stocks.shape[0]
d=stocks.shape[1]
r=np.zeros((N,d))
r=stocks.to_numpy()

In [62]:
def optimal_portfolio_SEG_unif(U,R,n_attempts=10**4,threshold=1/10**10):
    N=R.shape[0]
    d=R.shape[1]
    w_old=np.ones(d)/d
    w_new=np.zeros(d)
    attempt=0
    i=0
    while attempt<=n_attempts:
        i+=1
        k=np.random.randint(0,N)
        eta=np.random.rand()
        a=[w_old[j]*np.exp(eta*R[k,j]/(np.dot(w_old,R[k,:]))**(1-alpha)) for j in range(d)]
        w_new=a/np.sum(a)
        attempt+=1
        if U(w_new,R)>U(w_old,R)+threshold:
            w_old=w_new
            attempt=0
        if i%10000==0: print(s,i,100*w_old,U(w_old,R),eta)
        #print(i,100*w_old,U(w_old,R),eta) 
    return w_old, U(w_old,R), i

In [63]:
def refine(w,tol=1/10**3):
    d=w.shape[0]
    opt_num=[i for i in range(d) if w[i]>tol]
    not_opt_num=[i for i in range(d) if w[i]<=tol]
    w=w[opt_num]/np.sum(w[opt_num])    
    return opt_num, not_opt_num, w

In [64]:
def U(nu,r):
    if alpha==0:
        return np.mean(np.log(np.dot(r[0:N,:],nu)))
    else:
        return np.mean(np.dot(r[0:N,:],nu)**alpha)

In [ ]:
n_experiments=30
alpha=0.1
opt_portf=np.zeros((n_experiments,d))
opt_val=np.zeros(n_experiments)
n_iter=np.zeros(n_experiments)
opt_portf_refined=np.zeros
X=np.ones(n_experiments)
#Y=np.ones(n_experiments)
for s in range(n_experiments):
    np.random.seed(1+s)
    print(s)
    opt_portf[s,:], opt_val[s], n_iter[s]  = optimal_portfolio_SEG_unif(U,r)

0
0 10000 [5.15156582e-03 1.34066233e-03 8.62773995e-03 6.69350956e-03
 1.85818138e-03 1.63660018e-03 1.06053847e-03 3.89813314e-03
 6.76240639e-04 1.92641203e+01 4.07429836e-03 1.13904332e-03
 1.75661479e-02 2.72838427e-03 5.76824678e-03 2.10429549e-03
 8.06235344e+01 4.32805912e-03 4.36936725e-02] 1.0000754766663433 0.17595461032429316
0 20000 [3.45878318e-03 9.16215451e-04 5.95700175e-03 4.69953189e-03
 1.23030399e-03 1.15923287e-03 6.91057116e-04 2.66968203e-03
 4.77650656e-04 1.93500170e+01 3.13796912e-03 7.29704161e-04
 1.19263624e-02 1.89498373e-03 4.02451995e-03 1.47336004e-03
 8.05711636e+01 3.03645040e-03 3.13365918e-02] 1.0000754785716972 0.047511719795555685
0 30000 [2.61978584e-03 7.97695555e-04 5.17082169e-03 3.97029860e-03
 9.84129652e-04 9.39005989e-04 5.89986805e-04 2.28937394e-03
 4.12621384e-04 1.93512649e+01 2.88920135e-03 6.08470569e-04
 1.03684742e-02 1.52126609e-03 3.51430090e-03 1.29164783e-03
 8.05797174e+01 2.07375614e-03 2.89768920e-02] 1.0000754793237747 0.1

5 40000 [1.43702197e-03 8.23598189e-04 3.45662729e-03 3.54503485e-03
 9.60968524e-04 1.03957982e-03 7.61866997e-04 1.74484116e-03
 6.66901755e-04 1.92874653e+01 2.79860303e-03 8.54501102e-04
 7.24486725e-03 1.89604285e-03 3.18202312e-03 1.69160609e-03
 8.04399749e+01 1.50505416e-03 2.38950699e-01] 1.0000754831955765 0.01837610612906848
5 50000 [1.53595976e-03 1.01572198e-03 3.71532790e-03 5.08355896e-03
 1.19620863e-03 1.20018570e-03 7.37268660e-04 1.99067902e-03
 6.03659445e-04 1.76521572e+01 3.18703787e-03 1.08737889e-03
 9.24270008e-03 2.03413823e-03 2.97844865e-03 2.24540587e-03
 7.76921532e+01 1.81727929e-03 4.61601863e+00] 1.0000755196702418 0.536744043684117
5 60000 [1.50492453e-03 1.00960962e-03 3.71388973e-03 4.74684626e-03
 1.15155449e-03 1.17500848e-03 6.94922114e-04 1.76041638e-03
 5.84509691e-04 1.75852845e+01 2.89217306e-03 1.06631697e-03
 8.84926559e-03 1.94933227e-03 2.80268451e-03 2.18454147e-03
 7.76901919e+01 1.73303511e-03 4.68670453e+00] 1.0000755199048503 0.770752

10 30000 [1.33964554e-03 6.17689174e-04 5.40680896e-03 3.64945896e-03
 1.35560642e-03 9.46727732e-04 5.55998808e-04 2.42756996e-03
 6.79940324e-04 1.91968346e+01 2.99679636e-03 6.49044244e-04
 1.11188169e-02 1.19833195e-03 5.09164800e-03 1.71645709e-03
 8.06670678e+01 2.25821960e-03 9.40887762e-02] 1.0000754802513856 0.9957185689083802
10 40000 [1.23881146e-03 6.05298759e-04 5.21137110e-03 3.49314469e-03
 1.28836976e-03 9.45289985e-04 5.51644123e-04 2.32769178e-03
 6.66883481e-04 1.92152935e+01 2.82808645e-03 6.22876196e-04
 1.05909167e-02 1.14171151e-03 4.79936341e-03 1.65789676e-03
 8.06292979e+01 1.97767252e-03 1.15461572e-01] 1.000075480833841 0.652309130466411
10 50000 [1.08657119e-03 5.25599069e-04 4.69314843e-03 3.27882538e-03
 1.21558972e-03 8.14732860e-04 5.17087173e-04 2.15297740e-03
 5.90464493e-04 1.92170379e+01 2.40847753e-03 5.67907855e-04
 9.58699198e-03 1.00277592e-03 4.43348125e-03 1.48020170e-03
 8.05904209e+01 1.59297451e-03 1.56593352e-01] 1.000075481998779 0.099534

14 50000 [1.26092364e-03 6.40033420e-04 3.64002089e-03 3.26211278e-03
 9.88681374e-04 7.06979701e-04 6.30993199e-04 1.54501355e-03
 4.73257079e-04 1.93251462e+01 2.91517420e-03 7.88983886e-04
 6.41600925e-03 2.03122449e-03 3.14150753e-03 1.27919274e-03
 8.05858723e+01 1.67537078e-03 5.75860708e-02] 1.0000754804139627 0.17886243488894527
15
15 10000 [3.68067738e-03 1.07237507e-03 7.87071350e-03 5.74892727e-03
 1.30195615e-03 1.06414181e-03 1.09047479e-03 3.41694563e-03
 8.22558499e-04 1.93023011e+01 3.71005348e-03 1.10860848e-03
 1.85721533e-02 3.54570607e-03 7.24135943e-03 2.77019310e-03
 8.05980420e+01 4.81309969e-03 3.18269876e-02] 1.0000754768188715 0.12141095557112047
15 20000 [2.49963040e-03 7.04667177e-04 4.88513958e-03 3.95990726e-03
 9.16074117e-04 7.54212408e-04 7.34834157e-04 2.17010615e-03
 5.77795679e-04 1.93445659e+01 2.43471469e-03 6.94673571e-04
 1.17589798e-02 2.41551050e-03 4.78813883e-03 1.94446206e-03
 8.05879521e+01 3.30920466e-03 2.29340031e-02] 1.0000754788880257 

In [66]:
for s in range(n_experiments):
    opt_num, not_opt_num, w = refine(opt_portf[s,:])
    print(opt_val[s],opt_num,w,np.sum(opt_portf[s,opt_num]))  
    #X=1
    #Y=1
    print(X[s])
    for t in range(N):
        X[s]=X[s]*np.dot(w,r[t,opt_num])
    print(X[s])
#        Y[s]=Y[s]*np.dot(opt_portf[s,:],r[t,:])  

1.0000074402733303 [9, 16, 18] [0.20244644 0.793958   0.00359556] 0.9969093887739822
1.0
4051.4114808559084
1.0000074501072513 [9, 16, 18] [0.17659278 0.75046403 0.07294319] 0.9971511897467218
1.0
4100.718576641018
1.0000074394111003 [9, 16] [0.20307146 0.79692854] 0.9962782221607004
1.0
4046.344397665934
1.000007449500581 [9, 16, 18] [0.17589626 0.75232754 0.0717762 ] 0.9963862057062535
1.0
4100.571032307315
1.0000074394993521 [9, 12, 16, 18] [0.20141036 0.00123036 0.79388171 0.00347757] 0.9970445264450071
1.0
4051.3487100111356
1.0000074393906258 [9, 16, 18] [0.20106999 0.79618178 0.00274823] 0.9966638065880059
1.0
4050.121180789851
1.0000074498199385 [9, 16, 18] [0.17766921 0.74981204 0.07251875] 0.9967613624364268
1.0
4100.730141680498
1.0000074395158418 [9, 16, 18] [0.20265813 0.79504648 0.0022954 ] 0.9964817833466285
1.0
4049.6075626053794
1.0000074497860412 [9, 16, 18] [0.17845824 0.75322396 0.0683178 ] 0.9969191080079214
1.0
4100.287010030931
1.0000074504796426 [9, 16, 18] [0.1

In [67]:
stocks.columns[opt_num]

Index(['hp', 'morris', 'schlum'], dtype='object')

In [68]:
print(X,n_iter)

[4051.41148086 4100.71857664 4046.34439767 4100.57103231 4051.34871001
 4050.12118079 4100.73014168 4049.60756261 4100.28701003 4100.40405907
 4050.10872306 4100.06472104 4047.78417206 4048.14766347 4100.34915687
 4049.20180653 4100.07649717 4048.35043684 4049.30533655 4099.92046524
 4099.97960282 4100.4949999  4100.45424894 4049.05425407 4100.56527935
 4054.45423846 4100.57583584 4100.14786308 4100.2125266  4100.70941151] [59724. 41608. 40270. 52204. 34518. 41631. 38278. 53935. 51509. 44767.
 39985. 36217. 22904. 32518. 73237. 48951. 49797. 42370. 20763. 73464.
 30064. 80097. 58962. 44351. 46990. 52633. 55143. 37202. 75992. 53898.]


In [69]:
for s in range(n_experiments):
    print(s,X[s])

0 4051.4114808559084
1 4100.718576641018
2 4046.344397665934
3 4100.571032307315
4 4051.3487100111356
5 4050.121180789851
6 4100.730141680498
7 4049.6075626053794
8 4100.287010030931
9 4100.404059065903
10 4050.1087230559465
11 4100.064721039211
12 4047.7841720619167
13 4048.1476634666847
14 4100.349156867483
15 4049.201806525467
16 4100.076497173272
17 4048.35043683899
18 4049.305336546032
19 4099.920465243003
20 4099.979602822944
21 4100.494999896339
22 4100.454248940393
23 4049.0542540690853
24 4100.5652793504005
25 4054.4542384605443
26 4100.575835838145
27 4100.147863082211
28 4100.212526602083
29 4100.7094115062


In [70]:
n_iter

array([59724., 41608., 40270., 52204., 34518., 41631., 38278., 53935.,
       51509., 44767., 39985., 36217., 22904., 32518., 73237., 48951.,
       49797., 42370., 20763., 73464., 30064., 80097., 58962., 44351.,
       46990., 52633., 55143., 37202., 75992., 53898.])

In [71]:
z=np.zeros(4)
z=np.asarray([1,2,-1,5])
print(z[np.where(z>2)])

[5]


In [72]:
q=np.asarray([0.177,0.747,0.076])
print(q/np.sum(q))

[0.177 0.747 0.076]


In [73]:
np.sum(q)

0.9999999999999999

In [74]:
np.min(opt_portf,axis=0)[opt_num]

array([1.74052822e-01, 7.46841008e-01, 6.12825359e-04])

In [75]:
np.max(opt_portf,axis=0)[opt_num]

array([0.20397555, 0.79549497, 0.07273539])

In [76]:
n_iter

array([59724., 41608., 40270., 52204., 34518., 41631., 38278., 53935.,
       51509., 44767., 39985., 36217., 22904., 32518., 73237., 48951.,
       49797., 42370., 20763., 73464., 30064., 80097., 58962., 44351.,
       46990., 52633., 55143., 37202., 75992., 53898.])

In [77]:
print(n_iter.mean(),n_iter.min(),n_iter.max(),np.median(n_iter),n_iter.std())

47799.4 20763.0 80097.0 45878.5 14443.905482013282


In [78]:
jupyter notebook list

SyntaxError: invalid syntax (<ipython-input-78-46cbd787b5a9>, line 1)

In [86]:
s=np.argmax(opt_val)
print(s)
Y=1
for t in range(N):
    opt_num, not_opt_num, w = refine(opt_portf[s,:])    
    Y=Y*np.dot(w,r[t,opt_num])
print(stocks.columns[opt_num],Y)
w

9
Index(['hp', 'morris', 'schlum'], dtype='object') 4100.404059065903


array([0.17923866, 0.75181211, 0.06894922])